In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

!pip install laserembeddings==1.1.2
#!python -m laserembeddings download-models

from tqdm.auto import tqdm
import re
tqdm.pandas()
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from laserembeddings import Laser

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import sys
import os
os.system('python -m laserembeddings download-models')

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.7 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.35-py3-none-any.whl size=883989 sha256=e70b716ff631013c44a26d4821a642555252ae7d4abdc110369690c9043043c4
  Stored in directory: /root/.cache/pip/wheels/d1/ff/0e/e00ff1e22100702ac8b24e709551ae0fb29db9ffc843510a64
Successfully built sacremoses
  Attempting uninstall: sacremoses
    Found existing installation: sacremoses 0.0.53
    Uninstalling sacremoses-0.0.53:
      Successfully uninstalled sacremoses-0.0.53
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
allennlp 2.10.0 requires protobuf==3.20.0, but you have protobuf 3.19.4 which is incompatible.

✅   Downloaded https://dl.fbaipublicfiles.com/laser/mod

In [2]:
ls /opt/conda/lib/python3.7/site-packages/laserembeddings/data

93langs.fcodes  93langs.fvocab  bilstm.93langs.2018-12-26.pt


In [3]:
import site
loc = site.getsitepackages() 
data_path = loc[0]+'/laserembeddings/data'
print(site.getusersitepackages()) # String for user-specific package location)
loc[0]

/root/.local/lib/python3.7/site-packages


'/opt/conda/lib/python3.7/site-packages'

In [4]:
def similarity(v1, v2):
    n1 = np.linalg.norm(v1)
    n2 = np.linalg.norm(v2)
    return np.dot(v1, v2) / n1 / n2

def pairwise_euclidean_dists(x, y):
    dists = -2 * np.matmul(x, y.T)
    dists +=  np.sum(x**2, axis=1)[:, np.newaxis]
    dists += np.sum(y**2, axis=1)
    return  np.sqrt(dists)


laser = Laser()

In [5]:
sim_quran_torah = pd.read_csv('../input/quran-vs-bible/ensemble_preds_most_sim_pairs_quran_and_old_testament.csv')
corpus_emb = np.load('../input/multilingual-quran-bible-embeddings/mlt_quran_torah_emb.npy')
sim_quran_torah.head(1)

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score
0,Jeremiah 51:15,Jeremiah,51,15,"He hath made the earth by his power, he hath e...",The Bee,16,3,He hath created the heavens and the earth with...,93


In [6]:
def Multilingual_Quran_Bible_Search_Engine(query,size=10,language = 'en',metric = 'dot'):

    query_embedding = laser.embed_sentences(query, lang=language)

    if(metric == 'dot'):
        query_embedding = np.squeeze(np.asarray(query_embedding))
        linear_similarities = similarity(corpus_emb, query_embedding)
    else:
        linear_similarities = pairwise_euclidean_dists(corpus_emb, query_embedding)
        linear_similarities = np.squeeze(np.asarray(linear_similarities))
        linear_similarities = np.array(linear_similarities, dtype=np.float32)

    if(metric == 'dot'):
        Top_index_doc = linear_similarities.argsort()[:-(size+1):-1]
    else:
        Top_index_doc = linear_similarities.argsort()[:-(size+1):]
        Top_index_doc = Top_index_doc[:size]

  
    linear_similarities.sort()
    find = pd.DataFrame()
    for i,index in enumerate(Top_index_doc):
        find.loc[i,'t_citation'] = str(sim_quran_torah['t_citation'][index])
        find.loc[i,'t_book'] = str(sim_quran_torah['t_book'][index])
        find.loc[i,'t_chapter'] = str(sim_quran_torah['t_chapter'][index])
        find.loc[i,'t_verse'] = str(sim_quran_torah['t_verse'][index]) 
        find.loc[i,'t_text'] = str(sim_quran_torah['t_text'][index]) 
        
        find.loc[i,'q_Name'] = str(sim_quran_torah['q_Name'][index])
        find.loc[i,'q_Surah'] = str(sim_quran_torah['q_Surah'][index])
        find.loc[i,'q_Ayat'] = str(sim_quran_torah['q_Ayat'][index])
        find.loc[i,'q_Verse'] = str(sim_quran_torah['q_Verse'][index])
        find.loc[i,'similarity_score'] = str(sim_quran_torah['similarity_score'][index])

    for j,simScore in enumerate(linear_similarities[:-(size+1):-1]):
        find.loc[j,'Score'] = simScore
  
    return find

In [7]:
%%time
find = Multilingual_Quran_Bible_Search_Engine("who is near to allah?",size=2,language = 'en',metric = 'l2')

CPU times: user 381 ms, sys: 111 ms, total: 492 ms
Wall time: 368 ms


In [8]:
find

,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score,Score
0,Job 11:7,Job,11,7,Canst thou by searching find out God? canst th...,The Criterion,25,43,Hast thou seen him who chooseth for his god hi...,89,0.805662
1,Psalms 78:10,Psalms,78,10,"They kept not the covenant of God, and refused...",The Ascending Stairways,70,32,And those who keep their pledges and their cov...,78,0.782018


In [9]:
%%time
find = Multilingual_Quran_Bible_Search_Engine("আমি কি পুরস্কার পেতে পারি?",size=5,language = 'bn',metric = 'l2')
find

CPU times: user 233 ms, sys: 114 ms, total: 347 ms
Wall time: 119 ms


,t_citation,t_book,t_chapter,t_verse,t_text,q_Name,q_Surah,q_Ayat,q_Verse,similarity_score,Score
0,Psalms 35:12,Psalms,35,12,They rewarded me evil for good to the spoiling...,The Tidings,78,26,Reward proportioned (to their evil deeds).,76,0.787434
1,Job 28:12,Job,28,12,But where shall wisdom be found? and where is ...,Yâ Sîn,36,23,Shall I take (other) gods in place of Him when...,77,0.771004
2,Hosea 2:14,Hosea,2,14,"Therefore, behold, I will allure her, and brin...",The Calamity,101,10,"Ah, what will convey unto thee what she is! -",73,0.753783
3,Song of Solomon 8:3,Song of Solomon,8,3,"His left hand should be under my head, and his...",The Event,56,9,And (then) those on the left hand; what of tho...,80,0.740025
4,Proverbs 24:22,Proverbs,24,22,For their calamity shall rise suddenly; and wh...,The Calamity,101,3,"Ah, what will convey unto thee what the Calami...",71,0.715760
